# Transformation Equations:  ComCam <--> DES DR2

_Meagan N. Porter_

_2025.4.7_

## Initial Setup

In [ ]:
import numpy as np
import pandas as pd

import os
import sys
import glob
import math
import datetime

from collections import OrderedDict as odict

from astropy.io import fits
from astropy.table import Table

import fitsio

from scipy import interpolate
from scipy.optimize import leastsq

import healpy as hp

import plotly
from plotly.offline import download_plotlyjs, plot, iplot
import plotly.graph_objs as go

import matplotlib.pyplot as plt

%matplotlib inline

## Input Parameters

In [ ]:
# List of bands on which to run the transform fit...
#bandList = ['g']
bandList = ['g', 'r', 'i', 'z', 'Y']
            
# ComCam-->DES (toDES=True) or DES-->ComCam (toDES=False)...
#toDES = True
toDES = False


# Color to fit against...
color_name_1 = 'gi'

# Name of color_name_1 as the label in the QA plots...
colorLabel_1 = '(g-i)'


if toDES:
    # If ComCam-->DES, then the color to fit against and its plot label are ComCam...
    color_name_1 = """%s_ComCam""" % (color_name_1)
    colorLabel_1 = """$%s_{ComCam}$""" % colorLabel_1    
else:
    # If DES-->ComCam, then the color to fit against and its plot label are DES...
    color_name_1 = """%s_des""" % (color_name_1)
    colorLabel_1 = """$%s_{des}$""" % colorLabel_1    


# Order of polynomial fits...
norder = 2

# Sigma-clipping parameters...
nsigma = 3.0
niter = 3

# Identify directory containing the DES/SDSS match file...
matchDir = '/home/m/meaganp/DATA/thesis_work'

# Name of the DES/ComCam match file in matchDir
matchFile = 'cfds_comcam_match.fits'

# Base name of fit results output file...
if toDES:
    resultsFileBaseName = 'transFit.ComCam_to_des'
else:
    resultsFileBaseName = 'transFit.des_to_ComCam'

# Base name of QA plot output files...
if toDES:
    qaFileBaseName = 'qaPlot.ComCam_to_des.fit'
else:
    qaFileBaseName = 'qaPlot.des_to_ComCam.fit'

# Verbosity level (0, 1, 2, 3, ...)
verbose = 2

## Define Some Modules

In [ ]:
##################################
#
# Define some functions for fitting dmag vs. color...
#
# These functions are based on a scripts found at 
# http://linuxgazette.net/115/andreasen.html (by Anders Andreasen)
# and at
# http://www.phy.uct.ac.za/courses/python/examples/fitresonance.py (University of Cape Town)


##################################

# Parametric function:  
#  p is the parameter vector; 
#  For fp1, we assume a polynomial function in one color...
def fp1(p,color1_array):
    #retValue = p[0] + p[1]*color1_array + p[2]*color1_array*color1_array
    norder = p.size-1
    retValue = p[0]
    for i in range(norder):
        retValue = retValue + p[i+1]*color1_array**(i+1)
    return retValue


##################################

# Error function:
def residuals1(p,color1_array,dmag_array):
    err = (dmag_array-fp1(p,color1_array))
    return err


##################################

# Fitting code:
def transformFit1(color1_array, dmag_array, norder=2, verbose=0):

    # Calculate the median of dmag for use as an initial guess
    # for the overall zeropoint offset..
    mdn = np.median( dmag_array, None )

    # Parameter names
    #pname = (['c_0', 'c_1', 'c_2'])
    pname = []
    for i in range(0,norder+1):
        pname.append("""c_%d""" % i)

    # Initial parameter values
    #p0 = [mdn, 0.0, 0.0]
    p0 = (1+norder)*[0.0]
    p0[0] = mdn

    if verbose > 0:
        print() 
        print( 'Initial parameter values:  ', p0)

    # Perform fit

    p,cov,infodict,mesg,ier = leastsq(residuals1, p0, 
                                      args=(color1_array, dmag_array), 
                                      maxfev=10000, full_output=1)

    if ( ier>=1 and ier <=4):
        if verbose > 0:  print("Converged")
    else:
        # Add an exception error or a non-zero return value?
        print("Not converged")
        print(mesg)


    # Calculate some descriptors of the fit 
    # (similar to the output from gnuplot 2d fits)

    chisq=sum(infodict['fvec']*infodict['fvec'])
    dof=len(dmag_array)-len(p)
    rms=math.sqrt(chisq/dof)
    
    if verbose > 0:
        print("Converged with chi squared ",chisq)
        print("degrees of freedom, dof ", dof)
        print("RMS of residuals (i.e. sqrt(chisq/dof)) ", rms)
        print("Reduced chisq (i.e. variance of residuals) ", chisq/dof)
        print()


    # uncertainties are calculated as per gnuplot, "fixing" the result
    # for non unit values of the reduced chisq.
    # values at min match gnuplot
    perr = []
    if verbose > 0:  
        print("Fitted parameters at minimum, with 68% C.I.:")
    for i,pmin in enumerate(p):
        if verbose > 0:  
            print("%-10s %13g +/- %13g   (%5f percent)" % (pname[i],pmin,math.sqrt(cov[i,i])*math.sqrt(chisq/dof),
                                                           100.*math.sqrt(cov[i,i])*math.sqrt(chisq/dof)/abs(pmin)))
        perr.append(math.sqrt(cov[i,i])*math.sqrt(chisq/dof))
    if verbose > 0: print()

    if verbose > 0:
        print( "Correlation matrix:")
        # correlation matrix close to gnuplot
        print( "               ",)
        for i in range(len(pname)): print( "%-10s" % (pname[i],),)
        print()
        for i in range(len(p)):
            print( "%-10s" % pname[i],)
            for j in range(i+1):
                print( "%10f" % (cov[i,j]/math.sqrt(cov[i,i]*cov[j,j]),),)
            #endfor
            print()
        #endfor
        print()
        print()
        print()
    
    return p, perr, rms


In [ ]:
##################################

def createFitResultsOutputLine(norder, p, perr, rms, dmag_name, color1_name, color2_name=''):

    outputList = (2*(2*norder+1)+4)*[-9999.]
    outputList[0] = dmag_name
    outputList[1] = color1_name
    outputList[2] = color2_name
    for j in range(p.size):
        outputList[2*j+3] = p[j]
        outputList[2*j+4] = perr[j]
    outputList[2*(2*norder+1)+3] = rms
    outputLine = ','.join(map(str, outputList))
    return outputLine


##################################

def createFitResultsHeaderOutputLine(norder):

    outputList = (2*(2*norder+1)+4)*['c_']
    outputList[0] = 'dmag_name'
    outputList[1] = 'color1_name'
    outputList[2] = 'color2_name'
    for j in range(2*norder+1):
        outputList[2*j+3] = ("""c_%d""" % j)
        outputList[2*j+4] = ("""cerr_%d""" % j)
    outputList[2*(2*norder+1)+3] = 'rms'
    outputLine = ','.join(map(str, outputList))
    return outputLine


##################################

def transform1ColorQAPlots1(dmag, color1, res, norder, title, dmagName, colorLabel1, p, rms, outputFileName):

    # Prepare QA plots...
    #fig = plt.figure(figsize=(10,5))
    #fig = plt.figure(figsize=(40,20))
    fig = plt.figure(figsize=(20,10))
    fig.subplots_adjust(hspace=0.3)
    #fig.suptitle("This is a supertitle!")
    plt.rcParams.update({'font.size': 24})

   # We will exclude the lowest and highets 0.01% of color1, color2, 
    #  dmag, and residuals when plotting the QA figures...
    color1_desc = color1.describe(percentiles=[0.0001, 0.001, 0.01, 0.99, 0.999, 0.9999])
    dmag_desc = dmag.describe(percentiles=[0.0001, 0.001, 0.01, 0.99, 0.999, 0.9999])
    #res_desc = df.res.describe(percentiles=[0.0001, 0.001, 0.01, 0.99, 0.999, 0.9999])
    res_desc = res.describe(percentiles=[0.0001, 0.001, 0.01, 0.99, 0.999, 0.9999])
    #color1_min = color1_desc['1%']
    #color1_max = color1_desc['99%']
    #color1_min = color1_desc['min']
    #color1_max = color1_desc['max']
    #dmag_min = dmag_desc['1%']
    #dmag_max = dmag_desc['99%']
    #res_min = res_desc['1%']
    #res_max = res_desc['99%']
    color1_min = color1_desc['0.01%']
    color1_max = color1_desc['99.99%']
    dmag_min = dmag_desc['0.01%']
    dmag_max = dmag_desc['99.99%']
    res_min = res_desc['0.01%']
    res_max = res_desc['99.99%']

    # Plot 1:  Descriptive text...
    #plt.subplot(231)
    plt.subplot(221)
    if norder == 1:
        plot1Text = """%s \n\n%s = \n %.3f + \n %.3f*%s \n\n [rms: %.3f] \n\n [%.1f < %s < %.1f]""" % \
            (title, dmagName, p[0], p[1], colorLabel1, rms, color1_desc['min'], colorLabel1, color1_desc['max'])
    elif norder == 2:
        plot1Text = """%s \n\n%s = \n %.3f + \n %.3f*%s + \n %.3f*%s^2 \n\n [rms: %.3f] \n\n [%.1f < %s < %.1f]""" % \
            (title, dmagName, p[0], p[1], colorLabel1, p[2], colorLabel1, rms, color1_desc['min'], colorLabel1, color1_desc['max'])
    else:
        plot1Text = ''
    #plt.text(0.1,0.25,plot1Text)
    plt.text(0.1,0.00,plot1Text)
    plt.axis('off')

    
    # Plot 2:  2D hexbin histogram of dmag vs. color1...
    #plt.subplot(232) 
    plt.subplot(222)
    hb=plt.hexbin(color1, dmag, gridsize=100, bins='log', cmap='inferno')
    plt.axis([color1_min, color1_max, dmag_min, dmag_max])
    plt.xlabel(colorLabel1)
    plt.ylabel(dmagName)
    cb = fig.colorbar(hb)
    #cb.set_label('Number')
    cb.set_label('log10(N)')
    plt.grid(color='white')
    plt.grid(True)


    # Plot 3:  1d histogram of residuals...
    #plt.subplot(234) 
    plt.subplot(223) 
    #plt.hist(df.loc[:,'res'],bins=100)
    plt.hist(res,bins=100)
    plt.xlabel('residuals [mag]')
    plt.ylabel('Number')
    plt.grid(True)
    plt.grid(color='black')

    
    # Plot 4:  2d hexbin histogram of residuals vs. color1...
    #plt.subplot(235) 
    plt.subplot(224) 
    #hb = plt.hexbin(color1, df.loc[:,'res'], gridsize=100, cmap='inferno')
    hb = plt.hexbin(color1, res, gridsize=100, bins='log', cmap='inferno')
    plt.axis([color1_min, color1_max, res_min, res_max])
    plt.xlabel(colorLabel1)
    plt.ylabel('residuals [mag]')
    cb = plt.colorbar(hb)
    #cb.set_label('Number')
    cb.set_label('log10(N)')
    plt.grid(True)
    plt.grid(color='white')


    # Plot...
    plt.tight_layout()
    #plt.show()
    plt.savefig(outputFileName)

    return 0


##################################


## Read in Matched Catalog

In [ ]:
matchFile = os.path.join(matchDir,matchFile)
print( matchFile)

In [ ]:
# Check to make sure matchFile exists...
if os.path.isfile(matchFile)==False:
    print( """ERROR:  matchFile %s does not exist...""" % (matchFile))
if verbose > 0:
    print( 'matchFile: ', matchFile)


In [ ]:
tab = Table.read(matchFile, format='fits')
tab

In [ ]:
df = tab.to_pandas()
df

In [ ]:
#df['u_psfMag'] = -2.5*np.log10(df['u_psfFlux']) + 31.4
df['g_psfMag'] = -2.5*np.log10(df['g_psfFlux']) + 31.4
df['r_psfMag'] = -2.5*np.log10(df['r_psfFlux']) + 31.4
df['i_psfMag'] = -2.5*np.log10(df['i_psfFlux']) + 31.4
df['z_psfMag'] = -2.5*np.log10(df['z_psfFlux']) + 31.4
df['y_psfMag'] = -2.5*np.log10(df['y_psfFlux']) + 31.4

In [ ]:
#df['u_psfMagErr'] = 1.086*df['u_psfFluxErr']/df['u_psfFlux']
df['g_psfMagErr'] = 1.086*df['g_psfFluxErr']/df['g_psfFlux']
df['r_psfMagErr'] = 1.086*df['r_psfFluxErr']/df['r_psfFlux']
df['i_psfMagErr'] = 1.086*df['i_psfFluxErr']/df['i_psfFlux']
df['z_psfMagErr'] = 1.086*df['z_psfFluxErr']/df['z_psfFlux']
df['y_psfMagErr'] = 1.086*df['y_psfFluxErr']/df['y_psfFlux']

In [ ]:
# Rename columns...
df.rename(columns={'RA':'RA_des',
                   'DEC':'DEC_des',
                   'coord_ra':'RA_ComCam',
                   'coord_dec':'DEC_ComCam',
                   'MAG_STD_G':'g_des',
                   'MAG_STD_R':'r_des',
                   'MAG_STD_I':'i_des',
                   'MAG_STD_Z':'z_des',
                   'MAG_STD_Y':'Y_des',
                   'MAGERR_STD_G':'g_err_des',
                   'MAGERR_STD_R':'r_err_des',
                   'MAGERR_STD_I':'i_err_des',
                   'MAGERR_STD_Z':'z_err_des',
                   'MAGERR_STD_Y':'Y_err_des',
                   'g_psfMag':'g_ComCam',
                   'r_psfMag':'r_ComCam',
                   'i_psfMag':'i_ComCam',
                   'z_psfMag':'z_ComCam',
                   'y_psfMag':'Y_ComCam',
                   'g_psfMagErr':'g_err_ComCam',
                   'r_psfMagErr':'r_err_ComCam',
                   'i_psfMagErr':'i_err_ComCam',
                   'z_psfMagErr':'z_err_ComCam',
                   'y_psfMagErr':'Y_err_ComCam',
                  },inplace=True)

df.head(5)

## Add Columns to Matched Catalog Data Frame

In [ ]:
# Add color columns...
df.loc[:,'gr_des'] = df.loc[:,'g_des'] - df.loc[:,'r_des']
df.loc[:,'ri_des'] = df.loc[:,'r_des'] - df.loc[:,'i_des']
df.loc[:,'iz_des'] = df.loc[:,'i_des'] - df.loc[:,'z_des']
df.loc[:,'zY_des'] = df.loc[:,'z_des'] - df.loc[:,'Y_des']
df.loc[:,'gi_des'] = df.loc[:,'g_des'] - df.loc[:,'i_des']

df.loc[:,'gr_ComCam'] = df.loc[:,'g_ComCam'] - df.loc[:,'r_ComCam']
df.loc[:,'ri_ComCam'] = df.loc[:,'r_ComCam'] - df.loc[:,'i_ComCam']
df.loc[:,'iz_ComCam'] = df.loc[:,'i_ComCam'] - df.loc[:,'z_ComCam']
df.loc[:,'zY_ComCam'] = df.loc[:,'z_ComCam'] - df.loc[:,'Y_ComCam']
df.loc[:,'gi_ComCam'] = df.loc[:,'g_ComCam'] - df.loc[:,'i_ComCam']

In [ ]:
# Insert dmag column...
df.loc[:,'dmag'] = -9999.

## Create Initial Mask

In [ ]:
mask = ( ( df[color_name_1] > -1. ) & ( df[color_name_1] < 4.0 ) )

## Make Backup Copies of Initial Mask and Original Data Frame

In [ ]:
# Make a backup copy of original df...
df_orig = df.copy()

# Make a backup copy of original mask...
mask_orig = mask.copy()

## Run Fit in Each Filter Band

In [ ]:
# Create results output file...
resultsFile = """%s.dmag.%s.norder%d.csv""" % (resultsFileBaseName, color_name_1, norder)

# Open fit results output file...
try:
    fout = open(resultsFile, 'w')
except IOError:
    sys.exit('Unable to write to file ' + resultsFile)

# Write header to fit results output file...
hdr = createFitResultsHeaderOutputLine(norder)
fout.write(hdr+'\n')

for band in bandList:
    
    print() 
    print() 
    print() 
    print( "# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # ")
    print( band)
    print( "# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # ")
    print() 
    
    magName_des = """%s_des""" % (band)
    magErrName_des = """%s_err_des""" % (band)
    magName_ComCam = """%s_ComCam""" % (band)
    magErrName_ComCam = """%s_err_ComCam""" % (band)
    
    # Grab the original version of df from the backup copy...
    df = df_orig.copy()

    # Grab the original version of mask from the backup copy...
    mask = mask_orig.copy()

    # Update dmag column for {$band}_des - {$band}_ComCam...
    if toDES:
        df.loc[:,'dmag'] = df.loc[:,magName_des] - df.loc[:,magName_ComCam]
    else:
        df.loc[:,'dmag'] = df.loc[:,magName_ComCam] - df.loc[:,magName_des]
        
    # Update mask...
    mask1 = abs(df['dmag']) <= 10.
    mask2 = abs(df[magErrName_des]) <= 0.01
    mask3 = abs(df[magErrName_ComCam]) <= 0.02
    mask = mask & mask1 & mask2 & mask3

    # Iterate, with sigma-clipping...
    for i in range(niter):

        iiter = i + 1
        if verbose > 0:
            print( """   iter%d...""" % ( iiter ))

        # make a copy of original df, overwriting the old one...
        df = df[mask].copy()

        # Identify dmag and color1 series...
        dmag =  df.loc[:,'dmag']
        color1 = df.loc[:,color_name_1]

        # Perform fit...
        p,perr,rms = transformFit1(color1, dmag, norder, verbose)
        df.loc[:,'res'] = residuals1(p, color1, dmag)

        # Identify outliers...|
        stddev = df['res'].std()
        mask = (np.abs(df.res)< nsigma*stddev)


    # Output results to the results file...
    outputLine = createFitResultsOutputLine(2, p, perr, rms, band, color_name_1)
    fout.write(outputLine+'\n')
    
    # Create title/names for use in QA plots...
    if toDES:
        title = """$%s_{ComCam}$ --> $%s_{des}$""" % (band, band)
        dmagName = """$%s_{des} - %s_{ComCam}$""" % (band, band)
    else:
        title = """$%s_{des}$ --> $%s_{ComCam}$""" % (band, band)
        dmagName = """$%s_{ComCam} - %s_{des}$""" % (band, band)
    
    # Create QA plots...
    res =  df.loc[:,'res']
    outputFileName = """%s.dmag_%s.%s.norder%d.qa1.png""" % (qaFileBaseName, band, color_name_1, norder)
    status = transform1ColorQAPlots1(dmag, color1, res, norder, title, dmagName, colorLabel_1, p, rms, outputFileName)

fout.close()

In [ ]:
#%ls -FlthrG

In [ ]:
#%less transFit.ComCam_to_des.dmag.gi_ComCam.norder2.csv

In [ ]:
# ComCam-->DES:
# g vs. g-i (norder=1)
# r vs. r-i (norder=1)
# i vs. r-i (norder-1)
# z vs. r-i (norder=1)  [possibly z-Y instead]
# Y vs. r-i (norder=1)  [possibly i-z or g-i instead]

In [ ]:
# DES-->ComCam:
# g vs. g-i (norder=1)
# r vs. r-i (norder=1)
# i vs. r-i (norder-1)
# z vs. r-i (norder=1)  [possibly z-Y instead]
# Y vs. r-i (norder=1)  [possibly i-z or g-i instead]